In [1]:
import os
import hashlib
import secrets
import base64
import json
import sys
import requests
from Crypto.Util.number import long_to_bytes

Base_URL = "http://127.0.0.1:5000"

def generate_rsa_key_pair(bits=3072):       # Generate RSA key pair
    p = generate_prime(bits // 2)
    q = generate_prime(bits // 2)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = 65537
    d = pow(e, -1, phi_n)
    return {'N': n, 'e': e, 'd': d}

def generate_prime(bits):
    prime_candidate = 4
    while not is_prime(prime_candidate):
        prime_candidate = bytes_to_long(os.urandom(bits // 8))
        prime_candidate |= (1 << bits - 1) | 1
    return prime_candidate

def is_prime(n, k=40):      # Check if a number is prime
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False

    d = n - 1
    r = 0
    while d % 2 == 0:
        d //= 2
        r += 1

    for _ in range(k):
        a = secrets.randbelow(n - 2) + 2
        x = pow(a, d, n)
        if x != 1 and x != n - 1:
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == 1:
                    return False
                if x == n - 1:
                    break
            else:
                return False
    return True


def rsa_pss_pad(message, salt_len, key_size):   # RSA-PSS padding
    m_hash = hashlib.sha256(message).digest()
    em_len = key_size // 8 - 2 * len(m_hash) - 2
    if em_len < 0:
        raise ValueError("Message too long")
    salt = os.urandom(salt_len)
    m_prime = b'\x00\x01' + salt + m_hash
    padding = b'\x00' * (em_len - len(m_prime))
    return b'\x00\x00\x00\x00\x00\x00\x00\x00' + m_prime + padding


def sign_rsa_pss(private_key, message, salt_len=32):    # Sign message using RSA-PSS
    key_size = private_key['N'].bit_length()
    em = rsa_pss_pad(message, salt_len, key_size)
    m = bytes_to_long(em)
    s = pow(m, private_key['d'], private_key['N'])
    return long_to_bytes(s, key_size // 8)

def verify_rsa_pss(public_key, message, signature, salt_len=32):    # Verify RSA-PSS signature
    key_size = public_key['N'].bit_length()
    em = rsa_pss_pad(message, salt_len, key_size)
    s = bytes_to_long(signature)
    m = pow(s, public_key['e'], public_key['N'])
    em_prime = long_to_bytes(m, key_size // 8)
    return em == em_prime

def json_to_cookie(j: str) -> str:
    json_as_bytes = j.encode()
    base64_as_bytes = base64.b64encode(json_as_bytes, altchars=b'-_')
    base64_as_str = base64_as_bytes.decode()
    return base64_as_str

# Main function for demonstration
def main():
    key_pair = generate_rsa_key_pair()

    message = b"You got a 12 because you are an excellent student! :)"

    signature = sign_rsa_pss(key_pair, message)

    message_hex = message.hex()
    signature_hex = signature.hex()
    json_data = json.dumps({'msg': message_hex, 'signature': signature_hex})

    cookie = json_to_cookie(json_data)

    url = f"{Base_URL}/quote/"
    response = requests.get(url, cookies={'grade': cookie})
    print(response.text)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'requests'